In [2]:
import scanpy
import pandas as pd
import numpy as np
import bbknn
import matplotlib.pyplot as plt

In [5]:
bbknn

<module 'bbknn' from '/PHShome/ik936/anaconda3/lib/python3.7/site-packages/bbknn/__init__.py'>

In [ ]:
# fields_load <- c('meta_data', 'umap_after', 'umap_before', 'V', 'var_genes', 'exprs_raw')
# obj <- map(fields_load, function(name) {
#     readRDS(file.path(dname, paste0(name, '.rds')))
# })
# names(obj) <- fields_load



In [2]:
V=pd.read_table('/data/srlab/ik936/Roche/data/cache/obj_fibroblasts/V.tsv')

In [3]:
m=pd.read_csv('/data/srlab/ik936/Roche/data/cache/obj_fibroblasts/meta.csv')

/PHShome/ik936/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Tissues

In [117]:
%%time
res=bbknn.bbknn_pca_matrix(np.asarray(V), m.Tissue)


CPU times: user 12.1 s, sys: 79 ms, total: 12.1 s
Wall time: 12.1 s


In [118]:
%%time
import umap
reducer=umap.umap_.UMAP(random_state=42)
U=reducer.fit(res[0])

CPU times: user 34min 48s, sys: 47.8 s, total: 35min 36s
Wall time: 3min 12s


In [178]:
np.savetxt(X=U.embedding_, fname='/data/srlab/ik936/Roche/data/bbknn_U_tissues.csv', delimiter=',')

# Case status

In [180]:
%%time
res=bbknn.bbknn_pca_matrix(np.asarray(V), m.Case)


CPU times: user 25.2 s, sys: 98 ms, total: 25.3 s
Wall time: 25.2 s


In [181]:
%%time
import umap
reducer=umap.umap_.UMAP(random_state=42)
U=reducer.fit(res[0])

CPU times: user 35min 7s, sys: 52.4 s, total: 36min
Wall time: 3min 15s


In [182]:
np.savetxt(X=U.embedding_, fname='/data/srlab/ik936/Roche/data/bbknn_U_case.csv', delimiter=',')

# Donor

In [183]:
m=pd.read_csv('/data/srlab/ik936/Roche/data/cache/obj_fibroblasts/meta_for_bbknn.csv')

/PHShome/ik936/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [184]:
%%time
res=bbknn.bbknn_pca_matrix(np.asarray(V), m.BatchID)


CPU times: user 2min, sys: 959 ms, total: 2min 1s
Wall time: 2min 1s


In [185]:
%%time
import umap
reducer=umap.umap_.UMAP(random_state=42)
U=reducer.fit(res[0])

CPU times: user 31min 20s, sys: 41.7 s, total: 32min 2s
Wall time: 2min 58s


In [186]:
np.savetxt(X=U.embedding_, fname='/data/srlab/ik936/Roche/data/bbknn_U_donor.csv', delimiter=',')

# Donor within tissue

In [24]:
adata = scanpy.read_h5ad('/data/srlab/ik936/Roche/data/cache/obj_fibroblasts_scanpy.h5ad')

In [25]:
tissues = ['SalivaryGland', 'Gut', 'Synovium'] ## Failed on Lung 

In [26]:
import umap
from umap import umap_

adatas=[]
for tissue in tissues: 
    print(tissue)
    
    ## Copy and preprocess
    bdata = adata[adata.obs.Tissue == tissue].copy()
    scanpy.pp.recipe_zheng17(bdata, n_top_genes=1000)
    scanpy.tl.pca(bdata)

    ## BBKNN
    scanpy.external.pp.bbknn(bdata, batch_key='LibraryID')
    
    ## UMAP
    reducer=umap.umap_.UMAP(random_state=42)
    U=reducer.fit(bdata.uns['neighbors']['connectivities'])
    bdata.obsm['X_umap']=U.embedding_
    
    ## Append and save
    adatas.append(bdata)
    

SalivaryGland


/PHShome/ik936/anaconda3/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/PHShome/ik936/anaconda3/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/PHShome/ik936/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: This location for 'connectivities' is deprecated. It has been moved to .obsp[connectivities], and will not be accesible here in a future version of anndata.


Gut


/PHShome/ik936/anaconda3/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/PHShome/ik936/anaconda3/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/PHShome/ik936/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: This location for 'connectivities' is deprecated. It has been moved to .obsp[connectivities], and will not be accesible here in a future version of anndata.


Synovium


/PHShome/ik936/anaconda3/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/PHShome/ik936/anaconda3/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/PHShome/ik936/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: This location for 'connectivities' is deprecated. It has been moved to .obsp[connectivities], and will not be accesible here in a future version of anndata.


In [31]:
%%time

for adata, outname in zip(adatas, tissues):
    fname_out='/data/srlab/ik936/Roche/data/cache/bbknn_donors_{}_umap.csv'.format(outname)
    np.savetxt(X=adata.obsm['X_umap'], fname=fname_out, fmt='%.5f', delimiter=',')
    
    fname_out='/data/srlab/ik936/Roche/data/cache/bbknn_donors_{}_meta.csv'.format(outname)
    adata.obs.to_csv(fname_out) 

CPU times: user 1.53 s, sys: 39 ms, total: 1.57 s
Wall time: 1.98 s


In [4]:
# scanpy.pl.umap(adata, color=['LibraryID', 'Case'])